## COVID-19 Ingestion Pipeline

### Step 1: get COVID-19 dataset and copy daily report files to GCP bucket

In [ ]:
bucket_name = "covid-19-csv"

In [ ]:
!git clone https://github.com/CSSEGISandData/COVID-19.git

In [ ]:
!gsutil cp COVID-19/csse_covid_19_data/csse_covid_19_daily_reports/*.csv gs://{bucket_name}/daily

### Step 2: load files into BQ (Start from here)

In [ ]:
dataset_id = "covid_19_staging"

In [ ]:
!bq --location=US mk --dataset {dataset_id}

In [ ]:
!bq --location=US load --skip_leading_rows=1 --allow_jagged_rows=true --source_format=CSV {dataset_id}.Cases \
gs://covid-19-csv/daily/*.csv \
state:STRING,country:STRING,last_update:STRING,confirmed:INTEGER,deaths:INTEGER,recovered:INTEGER,latitude:NUMERIC,longitude:NUMERIC

### Step 3: Validate load

In [ ]:
%%bigquery
select count(*) as total_cases from covid_19_staging.Cases

#### Q1: Retrieve a few sample records

In [ ]:
%%bigquery
select min(last_update) as oldest_timestamp, max(last_update) as newest_timestamp
from covid_19_staging.Cases

#### Q2: Describe table schema

#### Q3: List number of records per day

In [ ]:
%%bigquery
select last_update as timestamp, count(*) as number_of_records
from covid_19_staging.Cases
group by last_update
order by last_update 
limit 10